# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import re, os, csv, glob, json

import cassandra
import numpy as np
import pandas as pd

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/heng-bin/python_scripts/de_course/data_modeling/modeling_with_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [1]:
# This should make a connection to a Cassandra instance your local machine 
# To establish connection and begin executing queries, need a session
import csv
from cassandra_manager import CassandraConn

conn = CassandraConn()

Connect default keyspace


#### Create Keyspace

In [2]:
key_space = 'user_events_log'
replication_strategy = 'SimpleStrategy'
replication_factor = 1
conn.create_keyspace(key_space, replication_strategy, replication_factor)

#### Set Keyspace

In [3]:
conn.switch_key_space(key_space)

Switch keyspace from `default` to `user_events_log`


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Drop and create user_session_records to meet query1.

In [4]:
# Define the name of the table
table_name = 'sessions'

# Drop table if exists
drop_table_template = "drop table if exists {table_name}"
conn.session.execute(drop_table_template.format(table_name=table_name))

# Create table if not exists
create_table_stmt = """
    create table if not exists {table_name}(
        session_id bigint,
        item_in_session int,
        artist text,
        song_title text,
        song_length float,
        primary key (session_id, item_in_session)
    )
"""
conn.session.execute(create_table_stmt.format(table_name=table_name))

#### Insert all sessions data from csv file into database.

In [5]:
file = 'event_datafile_new.csv'
table_name = 'sessions'

# Read file and insert data into database.
with open(file, 'r', encoding='utf-8') as rf:
    csv_file = csv.reader(rf)
    next(csv_file) # Skip header
    for row in csv_file:
        insert_data = {
            'session_id': int(row[8]),
            'item_in_session': int(row[3]),
            'artist': row[0],
            'song_title': row[9],
            'song_length': float(row[5]),
        }
        conn.insert_data(table_name, insert_data)

#### Validate results by getting `artist_name`, `song_title` and `song_length` in the music app history during `session_id = 338` and `item_in_session = 4`. 

#### The query should be presented as following:

```sql
select artist_name, song_title, song_length 
from sessions 
where session_id = 338 and item_in_session = 4
```

In [6]:
table_name = 'sessions'
select_columns = ['artist', 'song_title', 'song_length']
where_condition = [
    {'column_name': 'session_id', 'operator': '=', 'value': 338},
    {'column_name': 'item_in_session', 'operator': '=', 'value': 4},
]
result = conn.select_query(table_name, where_condition, select_columns)

for r in result:
    print(r)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Drop and create users to meet query2.

In [16]:
# Define the name of the table
table_name = 'users'

# Drop table if exists
drop_table_template = "drop table if exists {table_name}"
conn.session.execute(drop_table_template.format(table_name=table_name))

# Create table if not exists
create_table_stmt = """
    create table if not exists {table_name}(
        user_id int,
        session_id bigint,
        item_in_session int,
        artist text,
        song_title text,
        first_name text,
        last_name text,
        primary key ((user_id, session_id), item_in_session)
    )
"""
conn.session.execute(create_table_stmt.format(table_name=table_name))

#### Insert all users data from csv file into database.

In [17]:
file = 'event_datafile_new.csv'
table_name = 'users'

# Read file and insert data into database.
with open(file, 'r', encoding='utf-8') as rf:
    csv_file = csv.reader(rf)
    next(csv_file) # Skip column names
    for row in csv_file:
        insert_data = {
            'user_id': int(row[-1]),
            'session_id': int(row[8]),
            'item_in_session': int(row[3]),
            'artist': row[0],
            'song_title': row[9],
            'first_name': row[1],
            'last_name': row[4],
        }
        conn.insert_data(table_name, insert_data)

#### Validate results by getting `artist`, `song_title`(sort by item_in_sesion), `first_name`, `last_name` which conditions of the query is `user_id = 10` and `session_id` = 182

#### The query should be presented as following:

```sql
select artist, song_title, fiest_name, last_name
from users
where user_id = 10 and session_id = 182
```

In [18]:
table_name = 'users'
select_columns = ['artist', 'song_title', 'first_name', 'last_name']
where_condition = [
    {'column_name': 'user_id', 'operator': '=', 'value': 10},
    {'column_name': 'session_id', 'operator': '=', 'value': 182},
]
result = conn.select_query(table_name, where_condition, select_columns)

for r in result:
    print(r)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


#### Drop and create song_listen_records to meet query3

In [19]:
# Define the name of table
table_name = 'song_listen_records'

# Drop table if exists
drop_table_template = "drop table if exists {table_name}"
conn.session.execute(drop_table_template.format(table_name=table_name))

# Create table if not exists
create_table_stmt = """
    create table if not exists {table_name}(
        song_title text,
        user_id int,       
        first_name text,
        last_name text,
        primary key (song_title, user_id)
    )
"""
conn.session.execute(create_table_stmt.format(table_name=table_name))

#### Insert all song_listen_records data from csv file into database¶

In [20]:
file = 'event_datafile_new.csv'
table_name = 'song_listen_records'

# Read file and insert data into database.
with open(file, 'r', encoding='utf-8') as rf:
    csv_file = csv.reader(rf)
    column_names = next(csv_file)
    for row in csv_file:
        insert_data = {
            'song_title': row[9],
            'user_id': int(row[-1]),
            'first_name': row[1],
            'last_name': row[4],
        }
        conn.insert_data(table_name, insert_data)                    

#### Validate results by getting `first_name` and `last_name` of the users who listened the song 'All Hands Against His Own'

#### The query should be presented as following:

```sql
select first_name, last_name
from song_listen_records
where song_title = 'All Hands Against His Own'
```

In [21]:
table_name = 'song_listen_records'
select_columns = ['first_name', 'last_name']
where_condition = {'column_name': 'song_title', 'operator': '=', 'value': 'All Hands Against His Own'},

result = conn.select_query(table_name, where_condition, select_columns)

for r in result:
    print(r)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [22]:
table_names = ['sessions', 'users', 'song_listen_records']
drop_table_template = 'drop table if exists {}'

for table_name in table_names:
    conn.session.execute(drop_table_template.format(table_name))
    print("Successfully drop `{}` table".format(table_name))

Successfully drop `user_session_records` table
Successfully drop `user_records` table
Successfully drop `song_user_records` table


### Close the session and cluster connection¶

In [24]:
# session.shutdown()
# cluster.shutdown()
conn.disconnect()